In [1]:
# Installation des dépendances
!pip install torch torchvision matplotlib pandas tqdm

In [2]:
# Imports des bibliothèques essentiels
import os
import random
import torch
import torchvision
from torchvision import transforms, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.datasets import ImageFolder

**torch** : pour le cœur du deep learning et du modèle ResNet-50.

**torchvision** : pour les jeux de données, les transformations d’images et les modèles pré-entraînés.

**matplotlib** : pour la visualisation (affichage d’images, de courbes, etc.).

**pandas** : pour manipuler les métadonnées (CSV du dataset CUB).

**tqdm** : pour les barres de progression pendant l’entraînement.

**os, random** : gestion des fichiers, chemins et génération pseudo-aléatoire (utile pour la reproductibilité).

**matplotlib.pyplot, PIL.Image** visualisation et affichage des images et prédictions.

In [3]:
# Vérification du device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Appareil utilisé : {device}")

# Création du répertoire principal
base_dir = "/content/CUB_200_2011"
if not os.path.exists(base_dir):
    os.makedirs(base_dir)
print(f"Répertoire de travail : {base_dir}")

Appareil utilisé : cuda
Répertoire de travail : /content/CUB_200_2011


In [4]:
import zipfile
import urllib.request

# URL officielle du dataset
url = "https://data.caltech.edu/records/65de6-vp158/files/CUB_200_2011.tgz"
dataset_path = "/content/CUB_200_2011.tgz"

# Téléchargement
if not os.path.exists(dataset_path):
    print("Téléchargement du dataset CUB_200_2011...")
    urllib.request.urlretrieve(url, dataset_path)
    print("Téléchargement terminé.")
else:
    print("Dataset déjà téléchargé.")

# Extraction
import tarfile

if not os.path.exists(os.path.join(base_dir, "images")):
    print("Extraction en cours...")
    with tarfile.open(dataset_path, "r:gz") as tar:
        tar.extractall(path="/content/")
    print("Extraction terminée !")
else:
    print("Dataset déjà extrait.")

# Vérification du contenu
print("\nContenu du dossier :")
print(os.listdir(base_dir)[:10])

⬇Téléchargement du dataset CUB_200_2011...
Téléchargement terminé.
Extraction en cours...


/tmp/ipython-input-2767401981.py:22: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path="/content/")


Extraction terminée !

Contenu du dossier :
['images.txt', 'bounding_boxes.txt', 'README', 'images', 'image_class_labels.txt', 'train_test_split.txt', 'parts', 'classes.txt', 'attributes']


# **Chargement et préparation des métadonnées du dataset CUB-200-2011**

In [5]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch
import os

# Dossier des images
images_root = "/content/CUB_200_2011/images"

# Charger les métadonnées : classes et labels
image_labels = pd.read_csv("/content/CUB_200_2011/image_class_labels.txt",
                           sep=" ", header=None, names=["image_id", "class_id"])
image_paths = pd.read_csv("/content/CUB_200_2011/images.txt", sep=" ", header=None, names=["image_id", "rel_path"])

metadata = pd.merge(image_paths, image_labels, on="image_id")
metadata.head()

,image_id,rel_path,class_id
0,1,001.Black_footed_Albatross/Black_Footed_Albatr...,1
1,2,001.Black_footed_Albatross/Black_Footed_Albatr...,1
2,3,001.Black_footed_Albatross/Black_Footed_Albatr...,1
3,4,001.Black_footed_Albatross/Black_Footed_Albatr...,1
4,5,001.Black_footed_Albatross/Black_Footed_Albatr...,1


Cette fusion prépare la base d’annotations unique utilisée par le Dataset PyTorch.

# **Personalisation du dataset pour PyTorch**

In [6]:
class CUBDataset(Dataset):
    def __init__(self, metadata, images_root, transform=None):
        self.metadata = metadata
        self.images_root = images_root
        self.transform = transform

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        img_path = os.path.join(self.images_root, self.metadata.iloc[idx]["rel_path"])
        image = Image.open(img_path).convert("RGB")
        label = self.metadata.iloc[idx]["class_id"] - 1

        if self.transform:
            image = self.transform(image)
        return image, label

`__init__ Initialisation du dataset`

**metadata** : le DataFrame fusionné que tu as créé précédemment (image_id, rel_path, class_id).

**images_root** : chemin du répertoire principal contenant toutes les images.

**transform** : ensemble d’opérations (redimensionnement, normalisation, etc.) appliquées à chaque image avant qu’elle ne soit renvoyée.

`__len__ Taille du dataset`

Ceci permet à PyTorch de connaître le nombre total d’échantillons disponibles. Il est aussi utilisé notamment par le DataLoader pour le découpage en batchs.

`__getitem__ Chargement d’un échantillon individuel`

**idx** : index d’un échantillon dans le DataFrame.

**img_path** : construit le chemin absolu de l’image en combinant le dossier racine et le chemin relatif.

**Image.open(...).convert("RGB")** : lecture de l’image et conversion en format RGB.

**label = class_id - 1** : conversion en index 0-based (Car PyTorch exige que les classes soient de 0 à N-1).

Application éventuelle des transformations (resize, normalization, etc.).

**Préparation des transformations d’images**

In [7]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

**Détail de chaque transformation :**

| Étape                    | Description                                                        | But                                                                     |
| ------------------------ | ------------------------------------------------------------------ | ----------------------------------------------------------------------- |
| `Resize((224, 224))`     | Redimensionne toutes les images à 224×224 pixels                   | Taille d’entrée standard pour ResNet50                                  |
| `RandomHorizontalFlip()` | Fait une symétrie horizontale aléatoire de l’image                 | Augmentation de données → rend le modèle robuste à l’orientation        |
| `ColorJitter(...)`       | Modifie aléatoirement la luminosité, le contraste et la saturation | Simule différentes conditions lumineuses                                |
| `ToTensor()`             | Convertit l’image PIL (0–255) en tenseur PyTorch (0–1)             | Format attendu par le modèle                                            |
| `Normalize(mean, std)`   | Normalise les canaux RGB                                           | Aligne les données avec celles du pré-entraînement de ResNet (ImageNet) |


**Séparation en train et validation**

In [9]:
batch_size = 16

# Séparation du train et de la validation
train_meta = metadata.sample(frac=0.8, random_state=42)
val_meta = metadata.drop(train_meta.index)

# Création des datasets PyTorch
train_dataset = CUBDataset(train_meta, images_root, transform=train_transform)
val_dataset = CUBDataset(val_meta, images_root, transform=val_transform)

# Création des DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f"Nombre d'images train: {len(train_dataset)}")
print(f"Nombre d'images val:   {len(val_dataset)}")

Nombre d'images train: 9430
Nombre d'images val:   2358


**batch_size 16** : pour que le modèle traite 16 images à la fois avant de faire une mise à jour des poids.

**Séparation en train et validation**

| Élément                  | Description                                               |
| ------------------------ | --------------------------------------------------------- |
| `frac=0.8`               | On prend 80% des données pour l’entraînement.             |
| `drop(train_meta.index)` | Les 20% restants deviennent le jeu de validation.         |
| `random_state=42`        | Graine aléatoire fixe → garantit la **reproductibilité**. |

**Création des DataLoaders**

| Paramètre       | Description                                                                |
| --------------- | -------------------------------------------------------------------------- |
| `batch_size=16` | Nombre d’images traitées simultanément                                     |
| `shuffle=True`  | Mélange les images à chaque epoch → évite l’ordre fixe                     |
| `num_workers=2` | Nombre de processus parallèles pour charger les données (accélère sur GPU) |


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Détection du device utilisé
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Appareil utilisé : {device}")

# Chargement du ResNet-50 pré-entraîné
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

# Adaptation de la couche finale au nombre de classes du dataset CUB
num_classes = 200
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Envoi du modèle sur le device
model = model.to(device)

# Critère et optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Fonction pour sauvegarder le checkpoint
def save_checkpoint(model, path="resnet50_cub.pth"):
    torch.save(model.state_dict(), path)
    print(f"Modèle sauvegardé dans {path}")

Appareil utilisé : cuda
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 134MB/s]


# **Entraînement du modèle**

In [11]:
from tqdm.notebook import tqdm

# Paramètres
num_epochs_head = 5      # Phase 1 : entraînement de la tête
num_epochs_full = 10     # Phase 2 : fine-tuning complet
batch_size = 16

# Boucle d’entraînement pour la tête (Phase 1)
print("Phase 1 : Entrainement de la tête")
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

for epoch in range(num_epochs_head):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs_head}]"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    print(f"Epoch [{epoch+1}/{num_epochs_head}] Loss: {running_loss/total:.4f} Top-1: {100*correct/total:.2f}%")

# Boucle d’entraînement complète (Phase 2)
print("Phase 2 : Entraînement complet du modèle")
for param in model.parameters():
    param.requires_grad = True

for epoch in range(num_epochs_full):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs_full}]"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    print(f"Epoch [{epoch+1}/{num_epochs_full}] Loss: {running_loss/total:.4f} Top-1: {100*correct/total:.2f}%")

# Sauvegarde du modèle
save_checkpoint(model, path="/content/resnet50_cub.pth")

Phase 1 : Entrainement de la tête


Epoch [1/5]:   0%|          | 0/590 [00:00<?, ?it/s]

Epoch [1/5] Loss: 4.8557 Top-1: 8.84%


Epoch [2/5]:   0%|          | 0/590 [00:00<?, ?it/s]

Epoch [2/5] Loss: 3.9416 Top-1: 29.46%


...

Epoch [8/10] Loss: 0.1851 Top-1: 95.31%


Epoch [9/10]:   0%|          | 0/590 [00:00<?, ?it/s]

Epoch [9/10] Loss: 0.1273 Top-1: 97.03%


Epoch [10/10]:   0%|          | 0/590 [00:00<?, ?it/s]

Epoch [10/10] Loss: 0.1443 Top-1: 96.29%
Modèle sauvegardé dans /content/resnet50_cub.pth


**model.train()** : active le mode entraînement (Dropout, BatchNorm actifs).

**optimizer.zero_grad()** : réinitialise les gradients avant chaque batch.

**outputs = model(images)** : passe les images dans le réseau.

**loss = criterion(outputs, labels)** : calcule la perte (CrossEntropy).

**loss.backward()** : rétropropagation du gradient.

**optimizer.step()** : met à jour les poids de la couche fc.

**outputs.max(1)** : donne la classe la plus probable pour chaque image.

**predicted.eq(labels)** : compare les prédictions aux labels réels.